In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
import kerastuner as kt
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
x_train.shape, x_test.shape
plt.imshow(x_train[0])
print(y_train[0])

In [ ]:
def create_model(hp):
  num_hidden_layers = 1
  num_units = 128
  dropout_rate = 0.2
  learning_rate = 0.01

  if hp:
    num_hidden_layers = hp.Choice('num_hidden_layers', values = [1,2,3])
    num_units = hp.Choice('num_units', values = [32,64,128])
    dropout_rate = hp.Float('dropout_rate', min_value = 0.1, max_value = 0.5)
    learning_rate = hp.Float('learning_rate', min_value = 0.0001, max_value = 0.01)

  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
  model.add(tf.keras.layers.Lambda(lambda x: x/255.0))
  for i in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_units, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
model = create_model(None)
model.summary()

In [ ]:
class CustomTuner(kt.tuners.BayesianOptimization):
  def run_trial(self, trial, *args, **kwargs):
    kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 32, 64, step = 32)
    super(CustomTuner, self).run_trial(trial, *args, **kwargs)
    return self.results[-1]

tuner = CustomTuner(
      create_model,
      objective = 'val_accuracy',
      max_trials = 20,
      directory = 'logs',
      project_name = 'fasion_mnist',
      overwrite = True
)

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(
    x_train, y_train,
    validation_data = (x_test, y_test),
    epochs = 5, verbose = False
)

In [ ]:
model = tuner.get_best_models(num_models = 1)[0]
model.summary()

In [ ]:
history = model.fit(x_train, y_train,
          validation_data = (x_test, y_test),
          epochs = 20, batch_size = 128,
          callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy, patience = 3')])